**Importing Libraries** 

In [ ]:
import tensorflow as tf 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns 

**Data preprocessing** 

Training Image Preprocessing 

In [ ]:
training_set = tf.keras.utils.image_dataset_from_directory(
    'train',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
training_set 

In [ ]:
for x,y in training_set:
    print(x,x.shape)
    print(y,y.shape)
    break 

**Building Model**

In [ ]:
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from tensorflow.keras.models import Sequential 

In [ ]:
model = Sequential()

**Building Convolution Layer**

In [ ]:
model.add(Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[128,128,3]))
model.add(Conv2D(filters=32,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=64,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=128,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=128,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=256,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Conv2D(filters=512,kernel_size=3,padding='same',activation='relu'))
model.add(Conv2D(filters=512,kernel_size=3,activation='relu'))
model.add(MaxPool2D(pool_size=2,strides=2))

In [ ]:
model.add(Dropout(0.25)) #to avoid overfitting 

In [ ]:
model.add(Flatten()) 

In [ ]:
model.add(Dense(units=1500,activation='relu'))

In [ ]:
model.add(Dropout(0.4))

In [ ]:
model.add(Dense(units=80,activation='softmax'))

**Compiling the Model**

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(
    learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

**Model Training**

In [ ]:
training_history = model.fit(x=training_set,validation_data=validation_set,epochs=10)

**Model Evaluation**

In [ ]:
#model evaluation on training set 
train_loss,train_acc = model.evaluate(training_set)

In [ ]:
print(train_loss,train_acc)

In [ ]:
#model evaluation on validation set 
val_loss,val_acc = model.evaluate(validation_set)

In [ ]:
print(val_loss,val_acc)

**Saving the model**


In [ ]:
model.save("trained_model.keras")

In [ ]:
training_history.history

In [ ]:
#Recording history in json 
import json 
with open("training_hist.json","w") as f:
    json.dump(training_history.history,f)

**Accuracy Visulization**

In [ ]:
training_history.history['accuracy']

In [ ]:
training_history.history['val_accuracy']

In [ ]:
epochs = [i for i in range(1,11)]
epochs
plt.plot(epochs,training_history.history['accuracy'],color='red',label='Training Accuracy')
plt.plot(epochs,training_history.history['val_accuracy'],color='blue',label='Validation Accuracy')
plt.xlabel("No of epochs")
plt.ylabel("Accuracy result")
plt.title("Visualization of Accuracy Result")
plt.show()

**Some other metrics**

In [ ]:
class_name = validation_set.class_names
class_name

In [ ]:
test_set = tf.keras.utils.image_dataset_from_directory(
    'valid',
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
)

In [ ]:
y_pred = model.predict(test_set)
y_pred,y_pred.shape

In [ ]:
predicted_categories = tf.argmax(y_pred,axis=1)

In [ ]:
predicted_categories

In [ ]:
true_categories = tf.concat([y for x,y in test_set],axis=0)
true_categories

In [ ]:
Y_true = tf.argmax(true_categories,axis=1)
Y_true

In [ ]:
from sklearn.metrics import classification_report 

In [ ]:
print(classification_report(Y_true,predicted_categories,target_names = class_name))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_true,predicted_categories)
cm.shape

**Confusion matrix Visualization**

In [ ]:
plt.figure(figsize=(40,40))
sns.heatmap(cm,annot=True,annot_kws={'size':16})
plt.xlabel('Predicted class',fontsize=30)
plt.ylabel('Actual Class',fontsize=30)
plt.title('Confusion Matrix',fontsize=34)
plt.show()